# Capstone Project

## Content

* [Introduction: Business Problem](#introduction)
[Data]
[Methodology]
[Results]
[Discussion]
[Conclusion]

## Introduction: Business Problem <a id="introduction"></a>

# Scrape the Data from Wikipedia

In [5]:
import pandas as pd
import numpy as np
import json
#from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
# get data
url = 'https://de.wikipedia.org/wiki/Liste_der_Stadtbezirke_und_Stadtteile_K%C3%B6lns'
df = pd.read_html(url)[1]

,Nr.,Name,Einw.,Fläche km²,Einw./ km²
0,1.0,Stadtbezirk Innenstadt,127.033,1639,7.751
1,101.0,Altstadt-Süd,27.071,236,11.469
2,102.0,Neustadt-Süd,37.719,350,13.343
3,103.0,Altstadt-Nord,17.960,246,7.301
4,104.0,Neustadt-Nord,28.720,283,8.213
5,105.0,Deutz,15.563,524,2.968
6,2.0,Stadtbezirk Rodenkirchen,100.936,5462,1.848
7,201.0,Bayenthal,8.473,128,6.600
8,202.0,Marienburg (Köln),5.636,305,1.848
9,203.0,Raderberg,5.625,84,6.727


In [3]:
# read only Postcode information
df_TO = df_TO[0]

In [4]:
# drop Not assigned 
df_TO = df_TO[df_TO.Borough != 'Not assigned']

In [5]:
df_TO["Neighbourhood"] = np.where(df_TO["Neighbourhood"] == "Not assigned",
                                                  df_TO["Borough"],
                                                  df_TO["Neighbourhood"])

In [6]:
# Join Neighbourhoods with same Postcode
df_TO = df_TO.groupby(
    ["Postcode", "Borough"])["Neighbourhood"].apply(', '.join).reset_index()
df_TO.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_TO.shape

(103, 3)

# Append Locations to DataFrame

In [8]:
# import Location DataFrame
df_Loc = pd.read_csv("https://cocl.us/Geospatial_data")
df_Loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Merge DataFrame by Key
df_TOL = pd.merge(df_TO, df_Loc, left_on='Postcode', right_on='Postal Code')

In [10]:
# Drop Postal Code from Location DataFrame
df_TOL.drop(["Postal Code"], axis=1, inplace=True)

In [98]:
df_TOL.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_TOL.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Exploration and clustering the neighborhoods in Toronto

In [29]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'EOMLONZWTQLDDO0F1XWZR0ZDXTD2TRP03SPOGXWJW4Y5XYNZ' # your Foursquare ID
CLIENT_SECRET = 'SSQCGXCJLRR1I1ZKAUHUQOX54IVVADF1JWXXJOFGJPA5FANJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [33]:
neighborhood_latitude = df_TOL.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_TOL.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_TOL.loc[0, 'Neighbourhood'] # neighborhood name

In [43]:
# First 100 venues function

limit=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
# Create DataFrame with venues
TO_venues = getNearbyVenues(names=df_TOL['Neighbourhood'],
                            latitudes=df_TOL['Latitude'],
                            longitudes=df_TOL['Longitude']
                            )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

# Analyzing Neighorhoods

In [53]:
TO_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [68]:
# one hot encoding
TO_onehot = pd.get_dummies(TO_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TO_onehot['Neighborhood'] = TO_venues['Neighborhood'] 

# place Neighborhood to first column
neighb = TO_onehot["Neighborhood"]
TO_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
TO_onehot.insert(0, 'Neighborhood', neighb)

TO_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
# group by mean
TO_grouped = TO_onehot.groupby('Neighborhood').mean().reset_index()
TO_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0


In [73]:
# Create DataFrame with 5 most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TO_grouped['Neighborhood']

for ind in np.arange(TO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Doner Restaurant,Dessert Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool


# Cluster the Data

In [133]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

TO_grouped_clustering = TO_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 4, 3, 3, 3, 3, 3, 3, 3])

In [134]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Doner Restaurant,Dessert Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool


In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TO_merged = df_TOL

# merge data
TO_merged = TO_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [136]:
TO_merged["Cluster Labels"] = TO_merged["Cluster Labels"]+1

In [137]:
TO_merged.groupby("Cluster Labels").count()

,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Cluster Labels,,,,,,,,,,
1.0,12,12,12,12,12,12,12,12,12,12
2.0,3,3,3,3,3,3,3,3,3,3
3.0,3,3,3,3,3,3,3,3,3,3
4.0,81,81,81,81,81,81,81,81,81,81
5.0,2,2,2,2,2,2,2,2,2,2


In [140]:
# drop NaN
TO_merged = TO_merged.dropna(subset=['Cluster Labels'])

# Create a map with Cluster

In [147]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = 43.651070
longitude = -79.347015

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Neighborhood'], TO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters